# Short Cut Solver

Do a hardcore example on the "padding technique" by [Oberman and Ruan]. Show that there can be cases when it is not globally optimal.

In [ ]:
# load libraries
from lib.header_notebook import *

# assume that CPLEX back-end is installed
import Solvers.ShortCutSolver as ShortCutSolver
import Solvers.ShortCutSolver_CPLEX as ShortCutSolver_CPLEX

%matplotlib inline

## Problem Setup

In [ ]:
# create two simple test measurs
# muY is uniform measure
# muX is uniform, except for one huge chunk of mass at the center of the image.
# this huge change in concentration will cause the problem

res=(25,25)
nRes=res[0]*res[1]
qMass=1.
imgX=np.ones(res,dtype=np.double)
imgX[res[0]//2,res[1]//2]+=qMass*nRes
imgY=np.ones(res,dtype=np.double)

# preprocessing (normalize masses, extract geometric positions of pixels)
(muX,posX)=OTTools.processDensity_Grid(imgX,totalMass=(qMass+1.)*nRes,constOffset=None)
(muY,posY)=OTTools.processDensity_Grid(imgY,totalMass=(qMass+1.)*nRes,constOffset=None)

In [ ]:
# visualize images
vmax=np.max(imgX)
fig=plt.figure(figsize=(8,4))
fig.add_subplot(1,2,1)
plt.imshow(imgX,vmax=vmax,vmin=0)
fig.add_subplot(1,2,2)
plt.imshow(imgY,vmax=vmax,vmin=0)
plt.show()

In [ ]:
# hierarchical solving is just for fun at this point. just used to generate some sub-optimal initialization
#     such that the sparse iterations will get stuck

# set up hierarchical partitions

# finest layer above image has 2^partitionDepth grid points per dimension, then one below is image
partitionDepth=2
# another partition parameter, to be discussed later
partitionChildMode=HierarchicalPartition.THPMode_Grid

# create partitions from point clouds & measures, export partitions already to c++ library for later use
(partitionX,pointerX)=HierarchicalPartition.GetPartition(posX,partitionDepth,partitionChildMode,imgX.shape, mu=muX,\
    signal_pos=True, signal_radii=False,clib=SolverCFC, export=True, verbose=False)

(partitionY,pointerY)=HierarchicalPartition.GetPartition(posY,partitionDepth,partitionChildMode,imgY.shape, mu=muY,\
    signal_pos=True, signal_radii=True,clib=SolverCFC, export=True, verbose=False)

pointerYpos=HierarchicalPartition.getSignalPointer(partitionY,"pos")
pointerYradii=HierarchicalPartition.getSignalPointer(partitionY,"radii", lBottom=partitionY.nlayers-2)


# for demonstration purposes: compute dense cost functions at all levels
p=2. # exponent for Euclidean distance
cList=HierarchicalPartition.GetHierarchicalCost(partitionX, partitionY,\
    lambda posx, posy : OTTools.getEuclideanCostFunction(posx,posy,p=p))

# print a few stats on the created problem
print("cells in partition x: ", partitionX.cardLayers)
print("cells in partition y: ", partitionY.cardLayers)
print("dimensions of dense costs: ",[c.shape for c in cList])

## Solving

### Multiscale Solving

In [ ]:
# the algorithm has a modular structure. different components can be combined to final algorithm

# refinement:
#     generate initial fine coupling support when doing a layer refinement
methodSetup_Refinement=ShortCutSolver.getMethodSetup_Refinement(pointerX,pointerY,SolverCFC)

# coupling handler:
#     data structure for handling cost function and coupling.
#     in this small example use naive dense data structures (so no memory will be saved, only run-time)
#     for a sparse data structure see other example files.

methodSetup_CouplingHandler=ShortCutSolver.getMethodSetup_CouplingHandler_SemiDense()

# solver for sparse sub-problems
#     in this example only use CPLEX solver (Lemon requires some more preprocessing of densities)
#     couplingHandlerType must match the coupling handler chosen above (nothing to worry about now)
#     initializeBases=True indicates that warm-starting the solver during iterations on same scale will be used.
methodSetup_SubSolver=ShortCutSolver_CPLEX.getMethodSetup_SubSolver_CPLEX(\
        couplingHandlerType=ShortCutSolver_CPLEX.CH_SemiDense,initializeBases=True)


# shielding
#     now choose re-implementation of "padding technique"
methodSetup_Shielding=ShortCutSolver.getMethodSetup_Shielding_Padding()

In [ ]:
# do multi-scale solving.
#     algorithm gets all the chosen methods above and combines them into full ShortCut solver.
#     solves successively from very coarse level to finest level
#     is configured in verbose mode. at each level a small report is printed
time1=datetime.datetime.now()
result=ShortCutSolver.MultiscaleSolver(partitionX, partitionY, cList,\
    methodSetup_Refinement, methodSetup_CouplingHandler, methodSetup_SubSolver, methodSetup_Shielding,\
    nLayerInitial=1,nLayerFinal=None,\
    Verbose=True,\
    maxSteps=100,collectReports=True,measureTimes=True,stepwiseAnalysis=False)
time2=datetime.datetime.now()
print(time2-time1)

#### Verify Shielding Condition and Optimality

In [ ]:
# extract final neighbourhood and support from solver
xVars=ShortCutSolver.SolverGetXVars(result[0]["pointer"])
xSupport=ShortCutSolver.SolverGetSupport(result[0]["pointer"])

In [ ]:
# verify (symmetrized) shielding condition

## do some preprocessing to gather required information

# generate neighbourhoods
neighboursXList=[None]
for i in range(1,partitionX.nlayers):
    neighboursXList.append(ShortCutSolver.GetGridNeighbours(partitionX.dims[i]))
neighboursYList=[None]
for i in range(1,partitionY.nlayers):
    neighboursYList.append(ShortCutSolver.GetGridNeighbours(partitionY.dims[i]))

nLayer=partitionX.nlayers-1
shielding_info=ShortCutSolver.VerifyShieldingDuplex(cList[nLayer],xVars[0],xVars[1],\
        neighboursXList[nLayer][0],neighboursXList[nLayer][1],\
        neighboursYList[nLayer][0],neighboursYList[nLayer][1],\
        xSupport[0],xSupport[1]\
        )

print("shielding condition test:", shielding_info[0])
print("missed shields:", shielding_info[1][0].shape[0])

In [ ]:
# test explicitly for violated constraints
constraint_info=ShortCutSolver.VerifyDualConstraints(cList[nLayer],\
    result[0]["result_subsolver"]["alpha"],result[0]["result_subsolver"]["beta"],1E-10)

print("constraint violation test:", constraint_info[0])
print("violated constraints:", constraint_info[1][0].shape[0])

#### Check Objective

In [ ]:
mu=result[0]["result_couplinghandler"]["mu"]
alpha=result[0]["result_subsolver"]["alpha"]
beta=result[0]["result_subsolver"]["beta"]

In [ ]:
# primal score
np.sum(mu*cList[-1])

In [ ]:
# dual score
np.sum(alpha*muX)+np.sum(beta*muY)

In [ ]:
# close this solver session
ShortCutSolver.SolverClose(result[0]["pointer"])

### Compare with full Shielding

In [ ]:
# reset shielding method
methodSetup_Shielding=ShortCutSolver.getMethodSetup_Shielding_Grid()

In [ ]:
# resolve (turn of verbosity now)
time1=datetime.datetime.now()
result=ShortCutSolver.MultiscaleSolver(partitionX, partitionY, cList,\
    methodSetup_Refinement, methodSetup_CouplingHandler, methodSetup_SubSolver, methodSetup_Shielding,\
    nLayerInitial=1,nLayerFinal=None,\
    Verbose=False,\
    maxSteps=100,collectReports=False,measureTimes=False,stepwiseAnalysis=False)
time2=datetime.datetime.now()
print(time2-time1)

In [ ]:
# re-do shielding test
xVars=ShortCutSolver.SolverGetXVars(result[0]["pointer"])
xSupport=ShortCutSolver.SolverGetSupport(result[0]["pointer"])
xMap=xSupport[0][xSupport[1][:-1]]

nLayer=partitionX.nlayers-1
shielding_info=ShortCutSolver.VerifyShielding(cList[nLayer],xVars[0],xVars[1],\
        neighboursXList[nLayer][0],neighboursXList[nLayer][1],xMap)

print("shielding condition test:", shielding_info[0])
print("missed shields:", shielding_info[1][0].shape[0])

In [ ]:
# re-do test of dual constraints
constraint_info=ShortCutSolver.VerifyDualConstraints(cList[nLayer],\
    result[0]["result_subsolver"]["alpha"],result[0]["result_subsolver"]["beta"],1E-10)

print("constraint violation test:", constraint_info[0])
print("violated constraints:", constraint_info[1][0].shape[0])

In [ ]:
# re-compute objective
mu=result[0]["result_couplinghandler"]["mu"]
alpha=result[0]["result_subsolver"]["alpha"]
beta=result[0]["result_subsolver"]["beta"]

In [ ]:
# primal score
np.sum(mu*cList[-1])

In [ ]:
# dual score
np.sum(alpha*muX)+np.sum(beta*muY)

### Cleaning Up

In [ ]:
ShortCutSolver.SolverClose(result[0]["pointer"])
SolverCFC.Close(pointerX)
SolverCFC.Close(pointerY)